In [3]:
from pdfminer_testing import convert_pdf_to_text
import os
import chromadb

In [4]:
# Iterate through all the PDFs in the directory and convert them to text.
# Return a list of the text from each PDF. Texts are returned in text list, 
# and the source of each text is returned in the sources list.
def get_text_from_pdfs(directory):
    text = []
    sources = [] # list of strings, each string is the name of a PDF
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            text_file= convert_pdf_to_text(directory + filename)[0]
            source_list = [f"{filename}_page: {i+1}" for i in range(len(text_file))]
            #print (f"source_list: {source_list}")
            text+=text_file
            sources+=source_list
    return text, sources

In [5]:
text, sources = get_text_from_pdfs("sample_files/") 
print(f"Number of texts: {len(text)}")
print(f"Number of sources: {len(sources)}")

File name: sample_files/1502.03167.pdf
File name: sample_files/2205.14135.pdf
Number of texts: 45
Number of sources: 45


In [ ]:
#print(text[3])

In [112]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection("test_collection")

In [111]:
#chroma_client.delete_collection("test_collection")

In [7]:
# create a list of sources dictionaries
sources_list = []
for source in sources:
    sources_list.append({"source": source})

# create a list of ids for the inserted documents
ids_list = []
for i in range(len(text)):
    ids_list.append(str(i+1))
print(f"Number of ids: {len(ids_list)}")

Number of ids: 45


In [ ]:
#print(sources_list)

In [ ]:
#print(ids_list)

In [ ]:
from sentence_transformers import SentenceTransformer


In [107]:
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

modules.json: 100%|██████████| 229/229 [00:00<?, ?B/s] 
C:\Users\rochakchadha\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rochakchadha\.cache\huggingface\hub\models--sentence-transformers--multi-qa-mpnet-base-dot-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
co

In [108]:
embeddings = model.encode(text)

In [109]:
print(embeddings.shape)

(45, 768)


In [42]:
#collection.delete(ids_list)

In [113]:
# insert the documents into the collection
collection.add(
    embeddings=embeddings,
    documents=text,
    ids=ids_list,
    metadatas=sources_list
)
#pretty_print(collection)

In [114]:
query = "Explain flash attention 2 in brief"

In [115]:
query_list = [query]
query_embeddings = model.encode(query_list)

In [135]:
n_results = 2

In [136]:
query_results  = collection.query(
    query_embeddings = query_embeddings,
    n_results=n_results,
    include= [ "documents"]
)

In [ ]:
print(query_results['documents'][0][0])

In [138]:
context = ''.join(query_results['documents'][0][i] for i in range(n_results))

In [ ]:
print(context)

In [94]:
import os
import openai

In [65]:
openai.api_key = "EMPTY"
openai.api_base = "http://172.29.40.249:8000/v1"

In [139]:
system_message = {
    "role": "system",
    "content": "You are a helpful, respectful and honest assistant. Answer the question only based on the provided context and nothing else. If you cannot answer the question, please say so."
    #"content": "You are a helpful, respectful and honest assistant. Answer the question in a helpful manner." 
}

In [140]:
content = f"{query} \nContext: {context}"
#content = f"{query}"
#truncate the content to 3996 words
#content = content[:3996]

In [141]:
print(len(content.split()))

2133


In [144]:
print(content)

Explain flash attention 2 in brief 
Context: Attention
GFLOPs
HBM R/W (GB)
Runtime (ms)
Standard FlashAttention
66.6
40.3
41.7
75.2
4.4
7.3
Figure 2: Left: Forward + backward runtime of standard attention and FlashAttention for GPT-2 medium
(seq. length 1024, head dim. 64, 16 heads, batch size 64) on A100 GPU. HBM access is the primary factor aﬀecting
runtime. Middle: Forward runtime of FlashAttention (seq. length 1024, head dim. 64, 16 heads, batch size 64) on
A100 GPU. Fewer HBM accesses result in faster runtime, up to a point. Right: The runtime (for seq. length 4K) of
block-sparse FlashAttention is faster than FlashAttention by a factor proportional to the sparsity.
asymptotically improve on HBM accesses over all SRAM sizes. Proofs are in Appendix C.
Theorem 2. Let 𝑁 be the sequence length, 𝑑 be the head dimension, and 𝑀 be size of SRAM with 𝑑 ≤ 𝑀 ≤ 𝑁 𝑑.
Standard attention (Algorithm 0) requires Θ(𝑁 𝑑 + 𝑁 2) HBM accesses, while FlashAttention (Algorithm 1)
requires Θ(𝑁 2𝑑2 𝑀 −1) HB

In [142]:
user_message = {"role": "user", "content": content}


In [143]:
messages = [system_message, user_message]
response = openai.ChatCompletion.create(
    model="Llama-2-7b-chat-hf",
    temperature=0.9,
    max_tokens=1024,
    messages=messages,
    stream=True,
)

for chunk in response:
    content = chunk["choices"][0]["delta"].get("content", "")
    print(content, end="", flush=True)
print()

APIError: Invalid response object from API: '{"object":"error","message":"This model\'s maximum context length is 4096 tokens. However, your messages resulted in 4739 tokens. Please reduce the length of the messages.","code":40303}' (HTTP response code was 400)